In [1]:
import os
import sys
import random
import math
import numpy as np
import pandas as pd
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
PIC_DIR = "/home/lazukav/Kaggle/VidSegm/"

Using TensorFlow backend.


In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
#     pass

config = InferenceConfig()
config.display()

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
def getEncodedPixels(image, x_min, y_min, shape):
    encoded = ""
    for y in range(len(image)):
        start, end, flag = 0, 0, True
        for e in range(len(image[y])):
            if image[y][e]:
                if flag:
                    start, flag = e, False
                end = e
        if start != 0:
            encoded += "{} {}|".format(((x_min + y) * shape[1]) + y_min + start, end - start + 1)
    return encoded

def getPixelCount(image):
    return np.unique(image, return_counts = True)[1][1]

def select_cvpr_classes(results, class_names):
    classes = {'car': 33, 'motorcycle': 34, 'bicycle': 35, 'person': 36, 'truck': 38, 'bus': 39, 'tricycle': 40}
    filter_class = lambda x: classes[class_names[x]] if class_names[x] in classes.keys() else 0
    results['class_ids'] = list(map(filter_class, results['class_ids']))
    return results

def getDataFromImage(name, dataFrame):
# def getDataFromImage(name):
    image = skimage.io.imread(os.path.join(PIC_DIR, name.color))
    results = model.detect([image], verbose = 0)[0]
    r = select_cvpr_classes(results, class_names)
    for i in range(len(r['class_ids'])):
        x_min, y_min, x_max, y_max = r['rois'][i]
        tmp = r['masks'][x_min:x_max, y_min:y_max, i]
        label = r['class_ids'][i]
        encodedPixels = getEncodedPixels(tmp, x_min, y_min, r['masks'].shape)
        pixelCount = getPixelCount(tmp)
        confidence = r['scores'][i]
        dataFrame = dataFrame.append({'ImageId': name.id,
                   'LabelId': label,
                   'Confidence': confidence,
                   'PixelCount': pixelCount,
                   'EncodedPixels': encodedPixels
                  }, ignore_index = True)
#         dataFrame = pd.DataFrame({'ImageId': name.id,
#                    'LabelId': label,
#                    'Confidence': confidence,
#                    'PixelCount': pixelCount,
#                    'EncodedPixels' :encodedPixels
#                   }, ignore_index = True)
    return dataFrame

testData = pd.read_csv(os.path.join(PIC_DIR, "test_name.csv"), sep=",")

submission = pd.DataFrame(columns = ['ImageId', 'LabelId', 'Confidence', 'PixelCount', 'EncodedPixels'])
for e in range(len(testData)):
    submission = getDataFromImage(testData.iloc[e], submission)
    print (e, end = ' ')

In [ ]:
submission.to_csv('submission.csv', index=None)